In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/Stations.csv")
data.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN


In [3]:
s = data["Daytime Routes"].str.split(expand=True).apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = "Route"
data = data.join(s)
data.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Route
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,N
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,W
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,N
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,W
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN,N


In [4]:
data = data[["Station ID", "Line", "Route", "Stop Name", "GTFS Latitude", "GTFS Longitude"]]
data.head()

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude
0,1,Astoria,N,Astoria-Ditmars Blvd,40.775036,-73.912034
0,1,Astoria,W,Astoria-Ditmars Blvd,40.775036,-73.912034
1,2,Astoria,N,Astoria Blvd,40.770258,-73.917843
1,2,Astoria,W,Astoria Blvd,40.770258,-73.917843
2,3,Astoria,N,30 Av,40.766779,-73.921479


In [5]:
# this needs to be fixed 
name_groups = data.groupby(["Stop Name", "Route"])["Stop Name"]
suffix = name_groups.cumcount()+1
repeats = name_groups.transform('size')

data["Unique Name"] = np.where(repeats > 1, data["Stop Name"] + suffix.map(str), data["Stop Name"])
data[data["Unique Name"] != data["Stop Name"]]

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name
31,32,4th Av,R,36 St,40.655144,-74.003549,36 St1
40,41,Broadway - Brighton,B,7 Av,40.677050,-73.972367,7 Av1
272,272,Queens Blvd,R,36 St,40.752039,-73.928781,36 St2
277,277,Queens Blvd,B,7 Av,40.762862,-73.981637,7 Av2
421,421,Lenox - White Plains Rd,5,Gun Hill Rd,40.877850,-73.866256,Gun Hill Rd1
424,424,Lenox - White Plains Rd,5,Pelham Pkwy,40.857192,-73.867615,Pelham Pkwy1
444,444,Dyre Av,5,Gun Hill Rd,40.869526,-73.846384,Gun Hill Rd2
445,445,Dyre Av,5,Pelham Pkwy,40.858985,-73.855359,Pelham Pkwy2


In [6]:
data[data["Stop Name"] == "7 Av"]

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name
40,41,Broadway - Brighton,B,7 Av,40.677050,-73.972367,7 Av1
40,41,Broadway - Brighton,Q,7 Av,40.677050,-73.972367,7 Av
240,240,6th Av - Culver,F,7 Av,40.666271,-73.980305,7 Av
240,240,6th Av - Culver,G,7 Av,40.666271,-73.980305,7 Av
277,277,Queens Blvd,B,7 Av,40.762862,-73.981637,7 Av2
277,277,Queens Blvd,D,7 Av,40.762862,-73.981637,7 Av
277,277,Queens Blvd,E,7 Av,40.762862,-73.981637,7 Av


In [7]:
data["Name_Loc"] = data["Stop Name"] + data["GTFS Latitude"].astype(str) + data["GTFS Longitude"].astype(str)
data["Unique ID"] = data["Name_Loc"].apply(lambda x : list(data["Name_Loc"].unique()).index(x))

In [8]:
data[data["Stop Name"] == "7 Av"]

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name,Name_Loc,Unique ID
40,41,Broadway - Brighton,B,7 Av,40.677050,-73.972367,7 Av1,7 Av40.67705-73.972367,40
40,41,Broadway - Brighton,Q,7 Av,40.677050,-73.972367,7 Av,7 Av40.67705-73.972367,40
240,240,6th Av - Culver,F,7 Av,40.666271,-73.980305,7 Av,7 Av40.666271-73.980305,238
240,240,6th Av - Culver,G,7 Av,40.666271,-73.980305,7 Av,7 Av40.666271-73.980305,238
277,277,Queens Blvd,B,7 Av,40.762862,-73.981637,7 Av2,7 Av40.762862-73.981637,275
277,277,Queens Blvd,D,7 Av,40.762862,-73.981637,7 Av,7 Av40.762862-73.981637,275
277,277,Queens Blvd,E,7 Av,40.762862,-73.981637,7 Av,7 Av40.762862-73.981637,275


In [9]:
data.head()

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name,Name_Loc,Unique ID
0,1,Astoria,N,Astoria-Ditmars Blvd,40.775036,-73.912034,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd40.775036-73.91203399999999,0
0,1,Astoria,W,Astoria-Ditmars Blvd,40.775036,-73.912034,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd40.775036-73.91203399999999,0
1,2,Astoria,N,Astoria Blvd,40.770258,-73.917843,Astoria Blvd,Astoria Blvd40.770258-73.917843,1
1,2,Astoria,W,Astoria Blvd,40.770258,-73.917843,Astoria Blvd,Astoria Blvd40.770258-73.917843,1
2,3,Astoria,N,30 Av,40.766779,-73.921479,30 Av,30 Av40.766779-73.921479,2


In [10]:
data[data["Stop Name"] == "7 Av"]

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name,Name_Loc,Unique ID
40,41,Broadway - Brighton,B,7 Av,40.677050,-73.972367,7 Av1,7 Av40.67705-73.972367,40
40,41,Broadway - Brighton,Q,7 Av,40.677050,-73.972367,7 Av,7 Av40.67705-73.972367,40
240,240,6th Av - Culver,F,7 Av,40.666271,-73.980305,7 Av,7 Av40.666271-73.980305,238
240,240,6th Av - Culver,G,7 Av,40.666271,-73.980305,7 Av,7 Av40.666271-73.980305,238
277,277,Queens Blvd,B,7 Av,40.762862,-73.981637,7 Av2,7 Av40.762862-73.981637,275
277,277,Queens Blvd,D,7 Av,40.762862,-73.981637,7 Av,7 Av40.762862-73.981637,275
277,277,Queens Blvd,E,7 Av,40.762862,-73.981637,7 Av,7 Av40.762862-73.981637,275


In [11]:
# unique_names = pd.Series(data["Unique Name"].unique()).sort_values()
# data["unique_ID"] = data["Unique Name"].apply(lambda x : unique_names.loc[unique_names == x].index[0])
# data.head()

In [12]:
data.loc[(data["Route"] == 'S') & (data["Line"] == "Franklin Shuttle"),"Route"] = "S - Franklin"
data.loc[(data["Route"] == 'S') & (data["Line"] == "Rockaway"),"Route"] = "S - Rockaway"
data.loc[(data["Route"] == 'S') & (data["Line"] == "Lexington - Shuttle"),"Route"] = "S - Lexington"

In [13]:
groups = data.groupby(["Route", "Line"])
routes_within_line = dict(dict())
for (route, line), group_data in groups:
    if not route in routes_within_line.keys():
        routes_within_line[route] = {line : group_data}
    else:
        routes_within_line[route][line] = group_data

In [14]:
for rt, val in routes_within_line.items():
    print(rt)
    print(list(val.keys()), "\n")


1
['Broadway - 7Av'] 

2
['Broadway - 7Av', 'Clark St', 'Eastern Pky', 'Lenox - White Plains Rd', 'Nostrand'] 

3
['Broadway - 7Av', 'Clark St', 'Eastern Pky', 'Lenox - White Plains Rd'] 

4
['Eastern Pky', 'Jerome Av', 'Lexington Av'] 

5
['Dyre Av', 'Eastern Pky', 'Jerome Av', 'Lenox - White Plains Rd', 'Lexington Av', 'Nostrand'] 

6
['Lexington Av', 'Pelham'] 

7
['Flushing'] 

A
['8th Av - Fulton St', 'Liberty Av', 'Rockaway'] 

B
['6th Av - Culver', '8th Av - Fulton St', 'Broadway - Brighton', 'Concourse', 'Queens Blvd'] 

C
['8th Av - Fulton St'] 

D
['4th Av', '6th Av - Culver', '8th Av - Fulton St', 'Concourse', 'Queens Blvd', 'Sea Beach / West End / Culver / Brighton', 'West End'] 

E
['8th Av - Fulton St', 'Queens - Archer', 'Queens Blvd'] 

F
['63rd St', '6th Av - Culver', '8th Av - Fulton St', 'Broadway - Brighton', 'Queens Blvd', 'Sea Beach / West End / Culver / Brighton'] 

G
['6th Av - Culver', '8th Av - Fulton St', 'Crosstown'] 

J
['Jamaica', 'Queens - Archer'] 

L
['

In [15]:
line_orders = {
    '2' : [ 'Lenox - White Plains Rd',
            'Broadway - 7Av',
            'Clark St',
            'Eastern Pky',
            'Nostrand'
            ],
    '3' : [ 'Lenox - White Plains Rd',
            'Broadway - 7Av',
            'Clark St',
            'Eastern Pky'],
    '4' : [ 'Jerome Av',
            'Lexington Av',
            'Eastern Pky' ],
    '5' : [ 'Dyre Av',
            'Lenox - White Plains Rd',
            'Jerome Av',
            'Lexington Av', 
            'Eastern Pky',
            'Nostrand'],
    '6' : [ 'Pelham', 'Lexington Av' ],
    'A' : [ '8th Av - Fulton St',
            'Liberty Av',
            'Rockaway' ],
    'B' : [ 'Concourse',
            '8th Av - Fulton St',
            'Queens Blvd',
            '6th Av - Culver',
            'Broadway - Brighton',
            ], 
    'D' : [ 'Concourse',
            '8th Av - Fulton St', 
            '6th Av - Culver', 
            'Queens Blvd',
            '4th Av', 
            'West End',
            'Sea Beach / West End / Culver / Brighton' ],
    'E' : [ 'Queens - Archer',
            'Queens Blvd',
            '8th Av - Fulton St' ],
    'F' : [ 'Queens Blvd', 
            '63rd St',
            '8th Av - Fulton St',
            '6th Av - Culver',
            'Broadway - Brighton',
            'Sea Beach / West End / Culver / Brighton'],
    'G' : [ 'Crosstown', 
            '8th Av - Fulton St',
            '6th Av - Culver' ],
    'J' : [ 'Queens - Archer', 'Jamaica' ],
    'M' : [ 'Queens Blvd', 
            '6th Av - Culver', 
            'Jamaica', 
            'Myrtle Av' ],
    'N' : [ 'Astoria', 
            'Broadway - Brighton',
            'Manhattan Bridge', 
            '4th Av',
            'Sea Beach', 
           'Sea Beach / West End / Culver / Brighton' ],
    'Q' : [ 'Second Av', 
            '63rd St', 
            'Broadway - Brighton', 
            'Manhattan Bridge', 
            'Sea Beach / West End / Culver / Brighton' ],
    
    'R' : [ 'Queens Blvd',
            'Astoria', 
            'Broadway - Brighton',
            'Broadway',
            '4th Av' ],
    'W' : [ 'Astoria',
            'Broadway - Brighton',
            'Broadway' ],
    'Z' : [ 'Queens - Archer', 'Jamaica' ]
}

In [16]:
full_routes = dict()

single_line_routes = set(routes_within_line.keys()).difference(set(line_orders.keys()))
for rt in single_line_routes:
    key = list(routes_within_line[rt].keys())[0]
    full_routes[rt] = list(routes_within_line[rt][key]["Unique Name"])


for rt, lines in line_orders.items(): 
    full_routes[rt] = []
    for l in lines: 
        full_routes[rt] += list(routes_within_line[rt][l]["Unique Name"])

In [17]:
full_routes.keys()

dict_keys(['C', 'S - Rockaway', 'S - Franklin', '1', '7', 'L', 'S - Lexington', 'SIR', '2', '3', '4', '5', '6', 'A', 'B', 'D', 'E', 'F', 'G', 'J', 'M', 'N', 'Q', 'R', 'W', 'Z'])

In [18]:
data[data["Unique Name"] != data["Stop Name"]]

,Station ID,Line,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name,Name_Loc,Unique ID
31,32,4th Av,R,36 St,40.655144,-74.003549,36 St1,36 St40.655144-74.00354899999999,31
40,41,Broadway - Brighton,B,7 Av,40.677050,-73.972367,7 Av1,7 Av40.67705-73.972367,40
272,272,Queens Blvd,R,36 St,40.752039,-73.928781,36 St2,36 St40.752039-73.928781,270
277,277,Queens Blvd,B,7 Av,40.762862,-73.981637,7 Av2,7 Av40.762862-73.981637,275
421,421,Lenox - White Plains Rd,5,Gun Hill Rd,40.877850,-73.866256,Gun Hill Rd1,Gun Hill Rd40.87785-73.866256,419
424,424,Lenox - White Plains Rd,5,Pelham Pkwy,40.857192,-73.867615,Pelham Pkwy1,Pelham Pkwy40.857192-73.867615,422
444,444,Dyre Av,5,Gun Hill Rd,40.869526,-73.846384,Gun Hill Rd2,Gun Hill Rd40.869526-73.846384,442
445,445,Dyre Av,5,Pelham Pkwy,40.858985,-73.855359,Pelham Pkwy2,Pelham Pkwy40.858985-73.85535899999999,443


In [19]:
# borough hall needs to be moved above nevins st (route 4, 5)
old_loc = full_routes['4'].index("Borough Hall")
full_routes['4'].pop(old_loc)
new_loc = full_routes['4'].index("Nevins St")
full_routes['4'].insert(new_loc,'Borough Hall')

old_loc = full_routes['5'].index("Borough Hall")
full_routes['5'].pop(old_loc)
new_loc = full_routes['5'].index("Nevins St")
full_routes['5'].insert(new_loc, "Borough Hall")

# Rockaway 67 - Far rockaway needs to be swapped with 90th-116th (route A)
loc_90th = full_routes['A'].index("Beach 90 St")
for i in range(4):
    val = full_routes['A'][loc_90th]
    full_routes['A'].remove(val)
    full_routes['A'].append(val)
full_routes['A'] = full_routes['A'][:-4]

# 145th needs to be put in between 155th and 135th (route B)
# remove parkside av, beverly rd, cortelyou avenu h, av j, av m (route B)
full_routes['B'].remove("145 St")
full_routes['B'].insert(full_routes['B'].index("135 St"), "145 St")
full_routes['B'].remove("W 4 St-Wash Sq")
full_routes['B'].insert(full_routes['B'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")
full_routes['B'].remove("Parkside Av")
full_routes['B'].remove("Beverley Rd")
full_routes['B'].remove("Cortelyou Rd")
full_routes['B'].remove("Avenue H")
full_routes['B'].remove("Avenue J")
full_routes['B'].remove("Avenue M")
full_routes['B'].remove("Avenue U")
full_routes['B'].remove("Neck Rd")

# 145th needs to be put between 155th and 125th (route D)
# 7th ave after 59th st 
# wash square after 34th 
full_routes['D'].remove("145 St")
full_routes['D'].insert(full_routes['D'].index("125 St"), "145 St")
full_routes['D'].remove("7 Av")
full_routes['D'][full_routes['D'].index("W 4 St-Wash Sq")] = "7 Av"
full_routes['D'].insert(full_routes['D'].index('Broadway-Lafayette St'), "W 4 St-Wash Sq")

# ROUTE F
# 'Jay St-MetroTech' before bergen
# 'W 4 St-Wash Sq', before 'Broadway-Lafayette St'
full_routes['F'].remove("Jay St-MetroTech")
full_routes['F'].insert(full_routes['F'].index("Bergen St"), "Jay St-MetroTech")
full_routes['F'].remove("W 4 St-Wash Sq")
full_routes['F'].insert(full_routes['F'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")


# ROUTE M 
#  'W 4 St-Wash Sq', after 14th 
# reverse Myrtle - essex and metropolitan - central
full_routes['M'].remove("W 4 St-Wash Sq")
full_routes['M'].insert(full_routes['M'].index("Broadway-Lafayette St"), "W 4 St-Wash Sq")
reverse_1 = reversed(full_routes['M'][-13:-7])
reverse_2 = reversed(full_routes['M'][-7:])
full_routes['M'] = full_routes['M'][:-13]
full_routes['M'] += reverse_1
full_routes['M'] += reverse_2

# ROUTE N 
# queenseboro after 39th 
full_routes["N"].remove("Queensboro Plaza")
full_routes["N"].insert(full_routes["N"].index("Lexington Av/59 St"), "Queensboro Plaza")

# ROUTE Q
# canal after union square 
full_routes["Q"].remove("Canal St")
full_routes["Q"].insert(full_routes['Q'].index("DeKalb Av"), "Canal St")

# ROUTE R
# dekalb after jay street 
full_routes['R'].remove("DeKalb Av")
full_routes['R'].insert(full_routes['R'].index("Atlantic Av-Barclays Ctr"), "DeKalb Av")

# ROUTE W
# queensboro after 39th
full_routes["W"].remove("Queensboro Plaza")
full_routes["W"].insert(full_routes["W"].index("Lexington Av/59 St"), "Queensboro Plaza")

# ROUTE S 
# broken into separate shuttles

In [20]:
stations = data.drop("Line", axis=1)
stations.head()

,Station ID,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name,Name_Loc,Unique ID
0,1,N,Astoria-Ditmars Blvd,40.775036,-73.912034,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd40.775036-73.91203399999999,0
0,1,W,Astoria-Ditmars Blvd,40.775036,-73.912034,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd40.775036-73.91203399999999,0
1,2,N,Astoria Blvd,40.770258,-73.917843,Astoria Blvd,Astoria Blvd40.770258-73.917843,1
1,2,W,Astoria Blvd,40.770258,-73.917843,Astoria Blvd,Astoria Blvd40.770258-73.917843,1
2,3,N,30 Av,40.766779,-73.921479,30 Av,30 Av40.766779-73.921479,2


In [21]:
full_routes["S - Franklin"]

['Franklin Av', 'Park Pl', 'Botanic Garden']

In [22]:
edges = pd.DataFrame()
data_reindex = data.set_index(["Unique Name"])
edges.reindex(["Route", "To", "From"])
for key, val in full_routes.items():
    cur_route=pd.DataFrame()
    route = data_reindex[data_reindex["Route"] == key].loc[val].reset_index(drop=True)
    cur_route["To"] = route["Station ID"]
    cur_route["From"] = route["Station ID"]
    cur_route["Route"] = key
    cur_route["From"] = cur_route["From"].shift(-1)
    cur_route.dropna(inplace=True)    
    edges = edges.append(cur_route)

In [23]:
    
edges = pd.merge(
            edges, stations, left_on=["Route", "To"], right_on=["Route", "Station ID"]
        ).drop(["Station ID", "Stop Name"], axis=1
        ).rename({"GTFS Latitude" : "Latitude_1",
                  "GTFS Longitude" : "Longitude_1"},
                axis=1)

edges

,To,From,Route,Latitude_1,Longitude_1,Unique Name,Name_Loc,Unique ID
0,148,149.0,C,40.840719,-73.939561,168 St,168 St40.840719-73.939561,146
1,149,150.0,C,40.836013,-73.939892,163 St-Amsterdam Av,163 St-Amsterdam Av40.836013-73.939892,147
2,150,151.0,C,40.830518,-73.941514,155 St,155 St40.830518-73.941514,148
3,151,152.0,C,40.824783,-73.944216,145 St,145 St40.824783000000004-73.944216,149
4,152,153.0,C,40.817894,-73.947649,135 St,135 St40.817894-73.947649,150
...,...,...,...,...,...,...,...,...
729,102,103.0,Z,40.718315,-73.987437,Delancey St-Essex St,Delancey St-Essex St40.718315000000004-73.987437,101
730,103,104.0,Z,40.720280,-73.993915,Bowery,Bowery40.720279999999995-73.993915,102
731,104,105.0,Z,40.718092,-73.999892,Canal St,Canal St40.718092-73.999892,103
732,105,106.0,Z,40.713243,-74.003401,Chambers St,Chambers St40.713243-74.003401,104


In [24]:
edges = pd.merge(
            edges, stations, left_on=["Route", "From"], right_on=["Route", "Station ID"]
        ).drop(["Station ID", "Stop Name"], axis=1
        ).rename({"GTFS Latitude" : "Latitude_2",
                  "GTFS Longitude" : "Longitude_2"},
                axis=1)

edges["Distance"] = np.sqrt(
                        (edges["Latitude_1"]-edges["Latitude_2"])**2 +
                        (edges["Longitude_1"]-edges["Longitude_2"])**2
                    )

edges

,To,From,Route,Latitude_1,Longitude_1,Unique Name_x,Name_Loc_x,Unique ID_x,Latitude_2,Longitude_2,Unique Name_y,Name_Loc_y,Unique ID_y,Distance
0,148,149.0,C,40.840719,-73.939561,168 St,168 St40.840719-73.939561,146,40.836013,-73.939892,163 St-Amsterdam Av,163 St-Amsterdam Av40.836013-73.939892,147,0.004718
1,149,150.0,C,40.836013,-73.939892,163 St-Amsterdam Av,163 St-Amsterdam Av40.836013-73.939892,147,40.830518,-73.941514,155 St,155 St40.830518-73.941514,148,0.005729
2,150,151.0,C,40.830518,-73.941514,155 St,155 St40.830518-73.941514,148,40.824783,-73.944216,145 St,145 St40.824783000000004-73.944216,149,0.006340
3,151,152.0,C,40.824783,-73.944216,145 St,145 St40.824783000000004-73.944216,149,40.817894,-73.947649,135 St,135 St40.817894-73.947649,150,0.007697
4,152,153.0,C,40.817894,-73.947649,135 St,135 St40.817894-73.947649,150,40.811109,-73.952343,125 St,125 St40.811109-73.952343,151,0.008250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,102,103.0,Z,40.718315,-73.987437,Delancey St-Essex St,Delancey St-Essex St40.718315000000004-73.987437,101,40.720280,-73.993915,Bowery,Bowery40.720279999999995-73.993915,102,0.006769
730,103,104.0,Z,40.720280,-73.993915,Bowery,Bowery40.720279999999995-73.993915,102,40.718092,-73.999892,Canal St,Canal St40.718092-73.999892,103,0.006365
731,104,105.0,Z,40.718092,-73.999892,Canal St,Canal St40.718092-73.999892,103,40.713243,-74.003401,Chambers St,Chambers St40.713243-74.003401,104,0.005985
732,105,106.0,Z,40.713243,-74.003401,Chambers St,Chambers St40.713243-74.003401,104,40.710374,-74.007582,Fulton St,Fulton St40.710374-74.007582,105,0.005071


In [25]:
edges["To"] = edges["Unique ID_y"]
edges["From"] = edges["Unique ID_x"]



In [26]:
stations["Station ID"] = stations["Unique ID"]
stations.head()


,Station ID,Route,Stop Name,GTFS Latitude,GTFS Longitude,Unique Name,Name_Loc,Unique ID
0,0,N,Astoria-Ditmars Blvd,40.775036,-73.912034,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd40.775036-73.91203399999999,0
0,0,W,Astoria-Ditmars Blvd,40.775036,-73.912034,Astoria-Ditmars Blvd,Astoria-Ditmars Blvd40.775036-73.91203399999999,0
1,1,N,Astoria Blvd,40.770258,-73.917843,Astoria Blvd,Astoria Blvd40.770258-73.917843,1
1,1,W,Astoria Blvd,40.770258,-73.917843,Astoria Blvd,Astoria Blvd40.770258-73.917843,1
2,2,N,30 Av,40.766779,-73.921479,30 Av,30 Av40.766779-73.921479,2


In [27]:
edges[["Route", "To", "From", "Distance"]].to_csv("../data/edges.csv")
stations.drop_duplicates("Unique ID").drop(["Route", "Unique Name", "Unique ID", "Name_Loc"], axis=1).to_csv("../data/station_data.csv")